# A Parallel workflow model
Here we are attempting to create a parallel workflow model without using any LLM.

### 1. Import dependencies

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [ ]:
# For graphical visualization of the state graph
 
import nest_asyncio
nest_asyncio.apply()
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import Image

### 2. Create a state

In [ ]:
class BatsmanState(TypedDict):                      # The BatsmanState inherits a TypedDict
    runs: int
    balls: int
    fours: int
    sixes: int  
    strike_rate: float
    boundaries_per_ball: float
    boundary_percentage: float

### 3. Define the function for the graph node

In [ ]:
def calc_strike(state: BatsmanState) -> BatsmanState:
    runs = state["runs"]
    balls = state["balls"]

    strike = runs/balls

    state["strike_rate"] = strike

    return state

def calc_bound_rate(state: BatsmanState) -> BatsmanState:
    boundaries = state["fours"] + state["sixes"]
    balls = state["balls"]

    bound_per_ball = boundaries/balls

    state["boundaries_per_ball"] = bound_per_ball

    return state

def calc_bound_percent(state: BatsmanState) -> BatsmanState:
    boundaries = state["fours"] + state["sixes"]
    runs = state["runs"]

    bound_percent = (boundaries * 100)/runs

    state["boundary_percentage"] = bound_percent

    return state



### 4. Create the graph

In [ ]:
# Declare the graph:
graph = StateGraph(BatsmanState)

# Add a nodes/states to the graph:
graph.add_node("calc_strike", calc_strike)
graph.add_node("calc_bound_rate", calc_bound_rate)
graph.add_node("calc_bound_percent", calc_bound_percent)


# Add edges to the graph:
graph.add_edge(START, 'calc_strike')
graph.add_edge(START, 'calc_bound_rate')
graph.add_edge(START, 'calc_bound_percent')

graph.add_edge('calc_strike', END)
graph.add_edge('calc_bound_rate', END)
graph.add_edge('calc_bound_percent', END)

### 5. Compile the graph

In [ ]:
workflow = graph.compile()

### 6. Run the code

In [ ]:
initial_state = { "runs" : 50, "balls" : 12, "fours" : 2, "sixes" : 2  }

final_state = workflow.invoke(initial_state)                                # This will cause an issue as the parallel nodes try to update the state, even if they are editing different keys.

print(final_state)


In [ ]:
print(final_state["strike_rate"])

In [ ]:
print(final_state["boundaries_per_ball"])

In [ ]:
print(final_state["boundary_percentage"])

### 7. The Graph

In [ ]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER))